In [1]:
import pypsa
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from pypsa.plot import add_legend_circles

In [2]:
import pandas as pd

In [3]:
network = pypsa.Network()
network.set_snapshots(pd.date_range("2016-01-01 00:00", "2016-01-01 03:00", freq="H"))

network.add("Bus", "0", carrier="AC")
network.add("Bus", "0 gas", carrier="gas")

network.add("Carrier", "wind")
network.add("Carrier", "gas", co2_emissions=0.2)

network.add("GlobalConstraint", "co2_limit", sense="<=", constant=0.0)

network.add(
    "Generator",
    "wind turbine",
    bus="0",
    carrier="wind",
    p_nom_extendable=True,
    p_max_pu=[0.0, 0.2, 0.7, 0.4],
    capital_cost=1000,
)

network.add("Load", "load", bus="0", p_set=5.0)

network.add(
    "Link",
    "P2G",
    bus0="0",
    bus1="0 gas",
    efficiency=0.6,
    capital_cost=1000,
    p_nom_extendable=True,
)

network.add(
    "Link",
    "generator",
    bus0="0 gas",
    bus1="0",
    efficiency=0.468,
    capital_cost=400,
    p_nom_extendable=True,
)

network.add("Store", "gas depot", bus="0 gas", e_cyclic=True, e_nom_extendable=True)

In [4]:
network.generators


attribute,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,...,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
wind turbine,0,PQ,,0.0,True,0.0,inf,0.0,1.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0


In [5]:
network.plot()

c:\Users\carlo\anaconda3\envs\tesis1\lib\site-packages\pypsa\plot.py:217: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
  ax.axis(boundaries)


(<matplotlib.collections.PatchCollection at 0x26a942c35e0>,
 <matplotlib.collections.LineCollection at 0x26a942c3dc0>)

In [8]:

network.lopf(network.snapshots, pyomo=True)
network.objective

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation


AttributeError: _index

In [9]:
network.loads_t.p

Load
snapshot
2016-01-01 00:00:00
2016-01-01 01:00:00
2016-01-01 02:00:00
2016-01-01 03:00:00


In [10]:
network.links.p_nom_opt

Link
P2G          0.0
generator    0.0
Name: p_nom_opt, dtype: float64

In [11]:
network.stores.loc["gas depot"]

attribute
bus                     0 gas
type                         
carrier                   gas
e_nom                     0.0
e_nom_extendable         True
e_nom_min                 0.0
e_nom_max                 inf
e_min_pu                  0.0
e_max_pu                  1.0
e_initial                 0.0
e_initial_per_period    False
e_cyclic                 True
e_cyclic_per_period      True
p_set                     0.0
q_set                     0.0
sign                      1.0
marginal_cost             0.0
capital_cost              0.0
standing_loss             0.0
build_year                  0
lifetime                  inf
e_nom_opt                 0.0
Name: gas depot, dtype: object